# December 17th

Heat Loss We've got a grid again and we've got to learn things about every square let's do this.
There's a path that goes from the start to the end minimizing heat loss but it would have too many straight line segments.

In [30]:
example = [
    "2413432311323",
    "3215453535623",
    "3255245654254",
    "3446585845452",
    "4546657867536",
    "1438598798454",
    "4457876987766",
    "3637877979653",
    "4654967986887",
    "4564679986453",
    "1224686865563",
    "2546548887735",
    "4322674655533"
]

examplepartoneresult = 102

102

In [31]:
using DataStructures
import Base: isless
using IJulia

struct Step
    heatloss::Int
    currentposition::Tuple{Int,Int}
    direction::Tuple{Int,Int}
    currentsteps::Int
    previous::Union{Step,Nothing}
end

function isless(a::Step, b::Step)
    return a.heatloss < b.heatloss
end

directions = Dict([
    (0, 1) => [(0, 1), (1, 0), (-1, 0)],
    (0, -1) => [(0, -1), (1, 0), (-1, 0)],
    (1, 0) => [(1, 0), (0, 1), (0, -1)],
    (-1, 0) => [(-1, 0), (0, 1), (0, -1)]
])

function addoptions(step::Step, grid::Matrix{Int}, candidates::BinaryMinHeap{Step}, maxsteps::Int, minsteps::Int)
    newdirections = step.currentsteps < minsteps ? [step.direction] : directions[step.direction]
    for newdirection in newdirections
        (ty, tx) = step.currentposition .+ newdirection
        steps = newdirection == step.direction ? step.currentsteps + 1 : 1

        if steps > maxsteps
            continue
        end

        if ty >= firstindex(grid, 1) && ty <= lastindex(grid, 1) && tx >= firstindex(grid, 2) && tx <= lastindex(grid, 2)
            push!(candidates, Step(
                step.heatloss + grid[ty, tx],
                (ty, tx),
                newdirection,
                steps,
                step
            ))
        end
    end
end

function printgrid(step::Step, grid::Matrix{Int})
    printing = map(n -> string(n), copy(grid))
    while !isnothing(step)
        printing[step.currentposition[1], step.currentposition[2]] = if step.direction == (-1, 0)
            "^"
        elseif step.direction == (1, 0)
            "v"
        elseif step.direction == (0, -1)
            "<"
        elseif step.direction == (0, 1)
            ">"
        else
            "S"
        end
        step = step.previous
    end
    for y in firstindex(printing, 1):lastindex(printing, 1)
        println(string(printing[y, :]...))
    end
end

function astar(grid, maxsteps, minsteps)
    candidates = BinaryMinHeap([
        Step(0, (firstindex(grid, 1), firstindex(grid, 2)), (1, 0), 0, nothing),
        Step(0, (firstindex(grid, 1), firstindex(grid, 2)), (0, 1), 0, nothing)
    ])
    visited = Set()
    while !isempty(candidates)
        c = pop!(candidates)
        key = (c.currentposition, c.direction, c.currentsteps)
        if key in visited
            continue
        end
        push!(visited, key)
        if c.currentposition == (lastindex(grid, 1), lastindex(grid, 2)) && c.currentsteps >= minsteps
            return c
        end
        addoptions(c, grid, candidates, maxsteps, minsteps)
    end
    return best
end

astar (generic function with 3 methods)

In [32]:
include("./aoc.jl")

function partone(contents)
    grid = map(v -> parse(Int, v), makegrid(contents))
    astar(grid, 3, 1).heatloss
end

partone(example) == 102

true

In [33]:
function parttwo(contents)
    grid = map(v -> parse(Int, v), makegrid(contents))
    astar(grid, 10, 4).heatloss
end

parttwo(example) == 94

true

## Results

In [34]:
execute(17, partone, parttwo)

638
